## 3.1 Implémentation des relations 1:1

### Relation 1:1 bidirectionnelle avec SQLAlchemy

In [1]:
from sqlalchemy import Column,  ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker,  relationship

# Moteur d'accès à la base sqlalchemy.sqlite
engine = create_engine('sqlite:///sqlalchemy.sqlite')

<img src="../Relation_1_1_bidirectionnelle.png">

<p id="declarations" style="font-size:120%">Déclaration des classes métier :</p>

In [2]:
# Classe de base
Base = declarative_base()
Base.metadata.bind = engine

# Définition des classes
class User(Base):
    __tablename__ = 'user'

    id = Column(Integer, primary_key=True)
    pseudo = Column(String(16), nullable=False)
    email = Column(String(255), nullable=False)
    pwd = Column(String(32), nullable=False)
    
    office_id = Column(Integer,ForeignKey("office.id"))
    office = relationship("Office", uselist=False, back_populates="user")

class Office(Base):
    __tablename__ = 'office'
    
    id = Column(Integer, primary_key=True)
    building = Column(String(32),nullable=False)
    floor = Column(Integer,nullable=False)
    room_id = Column(String(32),nullable=False)
    
    user=relationship("User", uselist=False, back_populates="office")

<p id="usage" style="font-size:120%">Usage :</p>

In [3]:
# Session
DBSession = sessionmaker(bind=engine)
session = DBSession()

# Récupération d'un utilisateur, avec son bureau
user = session.query(User).filter(User.pseudo == 'rde').one()
print(user.pseudo, user.office.room_id, user.office.user.pseudo)

rde 110 rde


In [4]:
# Récupération d'un bureau, avec son utilisateur
office1 = session.query(Office).filter(Office.id == 1).one()
print(office1.building, office1.floor, office1.room_id, office1.user.pseudo)

W1bis 1 110 rde


In [5]:
# Le changement de bureau d'un utilisateur modifie le nouveau bureau et le bureau initial
user = office1.user
office2 = session.query(Office).filter(Office.id == 2).one()
user.office = office2

print(office1.building, office1.floor, office1.room_id, office1.user)
print(office2.building, office2.floor, office2.room_id, office2.user.pseudo)

session.add(user)
session.commit()

W1bis 1 110 None
rjh83 1 kijmdklt rde


In [6]:
# Le changement de l'occupant d'un bureau modifie l'occupant initial
user2 = session.query(User).filter(User.id == 2).one()

office2.user = user2
print(user.pseudo,user.office)
print(user2.pseudo,user2.office.id)

session.add(office2)
session.commit()

rde None
nxq 2


In [7]:
user.office = office1
session.add(user)
session.commit()

# Exemple de cours
user = session.query(User).filter(User.pseudo == 'rde').one()
office1 = user.office

# Le changement de bureau d'un utilisateur modifie le nouveau bureau et le bureau initial
office2 = Office(building='Deck',floor=1,room_id='RC 10')
user.office = office2

assert office1.user == None
assert office2.user == user